# 30 days of ML @ Kaggle

## About the data

The dataset is used for this competition is synthetic (and generated using a CTGAN), but based on a real dataset. The original dataset deals with predicting the amount of an insurance claim. Although the features are anonymized, they have properties relating to real-world features.

For this competition, you will be predicting a continuous target based on a number of feature columns given in the data. All of the feature columns, cat0 - cat9 are categorical, and the feature columns cont0 - cont13 are continuous.

Files:
- train.csv - the training data with the target column
- test.csv - the test set; you will be predicting the target for each row in this file
- sample_submission.csv - a sample submission file in the correct format


## Inital setup

In [1]:
import pandas as pd

train_path = 'data/train.csv'
test_path = 'data/test.csv'

# All features + target
raw_train = pd.read_csv(train_path)
# All features, no target
raw_test = pd.read_csv(test_path)

## Preprocessing 

In [11]:
# Remove rows with missing target, separate id and target from predictors
X_full = raw_train.dropna(axis=0, subset=['target'], inplace=False).copy()
y = X_full.target
X_full.drop(['id','target'], axis=1, inplace=True)

In [15]:
from sklearn.model_selection import train_test_split

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.75,
                                                                random_state=0)

In [19]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 16 and 
                    X_train_full[cname].dtype == "object"]

In [20]:
# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

In [21]:
# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

## Pipeline building

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the model
model = XGBRegressor(n_estimators=100, learning_rate=0.05)

# Bundle preprocessing and modeling code in a pipeline
full_pl = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
full_pl.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = full_pl.predict(X_valid)

print('RMSE:', mean_squared_error(y_valid, preds, squared=False))

# Submission generation 

In [ ]:
# Preprocessing of test data, fit model
preds_test = my_pipeline.predict(X_test) 

# Save test predictions to file
output = pd.DataFrame({'Id': raw_test.id,
                       'target': preds_test})
output.to_csv('submission.csv', index=False)

In [11]:
submission = pd.read_csv('data/submission.csv')

In [12]:
submission.head()

,Id,target
0,0,7.998531
1,5,8.405216
2,15,8.582426
3,16,8.405374
4,17,8.248979


In [13]:
submission_sample = pd.read_csv('data/sample_submission.csv')

In [14]:
submission_sample.head()

,id,target
0,0,0.5
1,5,0.5
2,15,0.5
3,16,0.5
4,17,0.5
